In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from scipy.ndimage.filters import uniform_filter1d
from matplotlib.gridspec import GridSpec
import geopandas as gpd
import xesmf as xe

from glob import glob

import matplotlib.path as mpath
import cartopy
import cartopy.crs as ccrs
crs_sp = ccrs.SouthPolarStereo(central_longitude=0)
crs_ll = ccrs.PlateCarree()
from cartopy.util import add_cyclic_point

import warnings
warnings.filterwarnings('ignore')

In [4]:
### --- convert longitude coordinates from 0-360 to -180-180 --- ###
def _convert_lon_to_180to180(ds, coord="lon"):
        ds = ds.copy()
        lon = ds[coord].values
        # Convert everything over 180 back to the negative (degrees W) values.
        lon[lon > 180] = lon[lon > 180] - 360
        # Need to account for clarifying dimensions if the grid is 2D.
        ds.coords[coord] = (ds[coord].dims, lon)
        return ds
def convert_lon(ds, coord="lon"):
        ds = _convert_lon_to_180to180(ds, coord=coord)
        # If 1-D, need to sort by lon (rearrange it) to allow it to be plotted with
        # xarray.
        if len(ds[coord].dims) == 1:
            ds = ds.sortby(coord)
        return ds
    
def get_model_components(model):
    #ds_year = model.groupby(model.time.dt.year)

    pr = model['pr'].groupby(model.time.dt.year).sum()
    pr = pr.sel(year = slice(2016, 2100))
    pr2, lon2 = add_cyclic_point(pr.values, coord = pr.lon.values)

    ws_annual= model['wind'].groupby(model.time.dt.year).mean()
    ws_annual = ws_annual.sel(year = slice(2016, 2100))
    ws_annual, lon2 = add_cyclic_point(ws_annual.values, coord = ws_annual.lon.values)

    summer = model.sel(time=model['time.season']=='DJF')
    summer['time'] = summer.time.values + np.timedelta64(31,'D')
    summer = summer.sel(time = slice('2016-01-01', '2100-01-01'))
    summer = summer.groupby(summer.time.dt.year).mean()
    #summer = summer.sel(year = slice(2016, 2100))
    ta_summer = summer['ta']- 273.15
    ta_summer, lon2 = add_cyclic_point(ta_summer.values, coord = ta_summer.lon.values)

    ta_annual = model['ta'].groupby(model.time.dt.year).mean() - 273.15
    ta_annual = ta_annual.sel(year = slice(2016, 2100))
    ta_annual, lon2 = add_cyclic_point(ta_annual.values, coord = ta_annual.lon.values)
    
    new_ds = xr.Dataset(
        data_vars=dict(
            pr=(["year", "lat", "lon"], pr2),
            wind=(["year", "lat", "lon"], ws_annual),
            ta=(["year", "lat", "lon"], ta_annual),
            ta_s=(["year", "lat", "lon"], ta_summer),
            
            #topo=(["lat", "lon"], model.topo.values),
            #land=(["lat", "lon"], model.land.values),
        ),
        coords=dict(
            year=pr.year.values,
            lat=model.lat.values,
            lon=lon2,
        ),)
    
    return new_ds

def get_change(model_file):
    model = xr.open_dataset(model_file)
    model = model.sel(time = slice('2015-01-01','2099-12-31'))
    model = convert_lon(model)
    new_ds = get_model_components(model)
    #new_ds_rolling = new_ds.rolling(year=10, center=True).mean(dim='year').dropna("year")

    regridder = xe.Regridder(new_ds, ds, "bilinear")
    ds_regrid = regridder(new_ds)

    #ds_anom = ds_regrid - ds_regrid.isel(year = 0)
    #ds_anom['topo'] = ds_regrid['topo']
    #ds_anom['land'] = ds_regrid['land']
    return ds_regrid

In [7]:
start_model = 'merra2'

if start_model=='era5':
    ds = xr.open_dataset('/glade/work/devond/ERA5/era_1985_2015.nc')
elif start_model=='merra2':
    ds = xr.open_dataset('/glade/work/devond/MERRA/merra_1985_2015.nc')

In [21]:
ssp5_model_files = glob(f'/glade/work/devond/FAC/ssp5/*.nc')
ssp3_model_files = glob(f'/glade/work/devond/FAC/ssp3/*.nc')
ssp1_model_files = glob(f'/glade/work/devond/FAC/ssp1/*.nc')

ssp5_m, ssp3_m, ssp1_m = list(), list(), list()

for item in ssp5_model_files:
    items = item.split('/')[-1]
    ssp5_m.append(items[:-8])
    
for item in ssp3_model_files:
    items = item.split('/')[-1]
    ssp3_m.append(items[:-8])

for item in ssp1_model_files:
    items = item.split('/')[-1]
    ssp1_m.append(items[:-8])
    
models = list(set(ssp5_m).intersection(ssp3_m).intersection(ssp1_m))
models.sort()

In [27]:
for scenario in ['ssp1','ssp3','ssp5']:


    i = 0

    for m in models:
        print(f'{m}, {i}/{len(models)}')
        m_file = f'/glade/work/devond/FAC/{scenario}/{m}_{scenario}.nc'

        anom = get_change(m_file)
        if i == 0: x = anom
        else: x = xr.concat([x, anom], "model")

        i=i+1

    x2 = x.assign_coords(model=("model", models))
    x2.to_netcdf(f'/glade/work/devond/FAC/netcdf_results/cmip6/{start_model}/{scenario}_change.nc')

ACCESS-CM2, 0/30
AWI-CM-1-1-MR, 1/30
BCC-CSM2-MR, 2/30
CAMS-CSM1-0, 3/30
CAS-ESM2-0, 4/30
CESM2, 5/30
CESM2-WACCM, 6/30
CMCC-CM2-SR5, 7/30
CMCC-ESM2, 8/30
CNRM-CM6-1, 9/30
CNRM-CM6-1-HR, 10/30
CNRM-ESM2-1, 11/30
CanESM5, 12/30
CanESM5-CanOE, 13/30
EC-Earth3, 14/30
EC-Earth3-Veg, 15/30
FGOALS-g3, 16/30
GISS-E2-1-G, 17/30
INM-CM4-8, 18/30
INM-CM5-0, 19/30
IPSL-CM6A-LR, 20/30
KACE-1-0-G, 21/30
MIROC-ES2L, 22/30
MIROC6, 23/30
MPI-ESM1-2-HR, 24/30
MPI-ESM1-2-LR, 25/30
MRI-ESM2-0, 26/30
NorESM2-MM, 27/30
TaiESM1, 28/30
UKESM1-0-LL, 29/30
ACCESS-CM2, 0/30
AWI-CM-1-1-MR, 1/30
BCC-CSM2-MR, 2/30
CAMS-CSM1-0, 3/30
CAS-ESM2-0, 4/30
CESM2, 5/30
CESM2-WACCM, 6/30
CMCC-CM2-SR5, 7/30
CMCC-ESM2, 8/30
CNRM-CM6-1, 9/30
CNRM-CM6-1-HR, 10/30
CNRM-ESM2-1, 11/30
CanESM5, 12/30
CanESM5-CanOE, 13/30
EC-Earth3, 14/30
EC-Earth3-Veg, 15/30
FGOALS-g3, 16/30
GISS-E2-1-G, 17/30
INM-CM4-8, 18/30
INM-CM5-0, 19/30
IPSL-CM6A-LR, 20/30
KACE-1-0-G, 21/30
MIROC-ES2L, 22/30
MIROC6, 23/30
MPI-ESM1-2-HR, 24/30
MPI-ESM1-2-LR, 